In [1]:
import cv2
import numpy as np
from fer import FER
import webbrowser
import random

# ------------- CONFIG -------------

# Emotion-wise YouTube playlists (change to your favourite links)
PLAYLISTS = {
    "happy": [
        "https://youtu.be/nJZcbidTutE?si=1kH9GbhD3NeIVUIr",
    ],
    "sad": [
        "https://youtu.be/sVRwZEkXepg?si=clkpCbzrGtBb1eGv",
    ],
    "angry": [
        "https://youtu.be/zqGW6x_5N0k?si=gkUTXcNSoreJo-_F",
    ],
    "neutral": [
        "https://youtu.be/ITyHqStTDeg?si=7TwM3y_2IfODchZb",
    ],
}

# Minimum confidence required to accept an emotion
CONFIDENCE_THRESHOLD = 0.80

# ----------------------------------


def map_raw_emotion_to_group(raw_emotion: str) -> str:
    """
    FER emotions: angry, disgust, fear, happy, sad, surprise, neutral
    We map them to 4 groups: happy, sad, angry, neutral.
    """
    raw_emotion = raw_emotion.lower()

    if raw_emotion == "happy":
        return "happy"
    if raw_emotion in ["sad", "fear"]:
        return "sad"
    if raw_emotion == "angry":
        return "angry"
    # disgust, surprise, neutral -> neutral
    return "neutral"


def pick_random_playlist_for_emotion(group_emotion: str) -> str:
    if group_emotion not in PLAYLISTS or not PLAYLISTS[group_emotion]:
        group_emotion = "neutral"
    return random.choice(PLAYLISTS[group_emotion])


def open_playlist(group_emotion: str):
    url = pick_random_playlist_for_emotion(group_emotion)
    print(f"[ACTION] Opening {group_emotion} playlist: {url}")
    webbrowser.open(url)


def main():
    detector = FER(mtcnn=True)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    print("Instructions:")
    print("  - Camera window me apna face clearly dikhana")
    print("  - Light thodi bright rakho")
    print("  - Jab ready ho (expression set kar liya ho) to 'c' dabao")
    print("  - ESC dabao bina capture ke exit karne ke liye\n")

    chosen_emotion = None
    chosen_confidence = 0.0
    chosen_group = None

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to read from camera.")
            break

        frame_resized = cv2.resize(frame, (640, 480))
        display_frame = frame_resized.copy()

        cv2.putText(
            display_frame,
            "Press 'c' to capture emotion, ESC to quit",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 255),
            2,
        )

        cv2.imshow("Emotion Capture", display_frame)

        key = cv2.waitKey(1) & 0xFF

        if key == 27:  # ESC
            print("Exiting without capturing emotion.")
            break

        if key == ord('c'):  # capture emotion
            # Take current frame and analyze
            rgb_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            result = detector.detect_emotions(rgb_frame)

            if not result:
                print("No face detected. Try again with your face closer/centered.")
                continue

            face = result[0]
            emotions_dict = face["emotions"]
            raw_emotion = max(emotions_dict, key=emotions_dict.get)
            score = emotions_dict[raw_emotion]

            print(f"Detected raw emotion: {raw_emotion} (confidence={score:.2f})")

            if score < CONFIDENCE_THRESHOLD:
                print("Confidence too low. Please try again with a clearer expression.\n")
                continue

            group_emotion = map_raw_emotion_to_group(raw_emotion)
            chosen_emotion = raw_emotion
            chosen_confidence = score
            chosen_group = group_emotion
            break

    # Close camera & window
    cap.release()
    cv2.destroyAllWindows()

    # If we got a valid emotion, open playlist
    if chosen_group is not None:
        print(f"\nFinal Emotion: {chosen_emotion} (group: {chosen_group}, conf={chosen_confidence:.2f})")
        open_playlist(chosen_group)
        print("Done. Program finished. 👍")
    else:
        print("No valid emotion captured. No song played.")


if __name__ == "__main__":
    main()

C:\Users\Piyush\anaconda3\envs\gestureenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Piyush\anaconda3\envs\gestureenv\lib\site-packages\fer\fer.py:38: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Instructions:
  - Camera window me apna face clearly dikhana
  - Light thodi bright rakho
  - Jab ready ho (expression set kar liya ho) to 'c' dabao
  - ESC dabao bina capture ke exit karne ke liye



C:\Users\Piyush\anaconda3\envs\gestureenv\lib\site-packages\keras\src\models\functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_1']
Received: inputs=Tensor(shape=(1, 64, 64))
  warnings.warn(msg)


Detected raw emotion: happy (confidence=0.55)
Confidence too low. Please try again with a clearer expression.

Detected raw emotion: neutral (confidence=0.52)
Confidence too low. Please try again with a clearer expression.

Detected raw emotion: angry (confidence=0.26)
Confidence too low. Please try again with a clearer expression.

Detected raw emotion: angry (confidence=0.55)
Confidence too low. Please try again with a clearer expression.

Detected raw emotion: happy (confidence=0.90)

Final Emotion: happy (group: happy, conf=0.90)
[ACTION] Opening happy playlist: https://youtu.be/nJZcbidTutE?si=1kH9GbhD3NeIVUIr
Done. Program finished. 👍
